In [1]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import tensorflow as tf
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import tqdm
import torch.optim as optim
import torch.nn as nn
# Download necessary NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from torch.utils.data import Dataset, DataLoader

E0000 00:00:1734743927.532995      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1221 01:18:47.541259871      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1221 01:18:47.541273032      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1221 01:18:47.541276364      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1221 01:18:47.541278761      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1221 01:18:47.541281148      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# Initialize the tokenizer using NLTK
tokenizer = word_tokenize

# Custom stopwords
stop_words = set(stopwords.words('english'))
negative_words = set(["a", "an", "no", "not", "don't", "never", "none", "nothing", "neither", "nowhere", "nobody", "n't"])

# Remove negative words from stopwords
stop_words -= negative_words

# Preprocessing function to clean and tokenize sentences
def preprocess_order(order):
    # Convert to lowercase
    order = order.lower()
    
    # Remove punctuation
    PUNCT_TO_REMOVE = string.punctuation
    order = order.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    
    # Remove extra spaces
    order = " ".join(order.split())
    
    # Tokenize the text using the NLTK tokenizer
    tokens = tokenizer(order)
    
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    return filtered_tokens

In [4]:
trainData = pd.read_json("/kaggle/input/PIZZA_train.json", lines=True)

In [5]:
partialtrainData = trainData.sample(frac=0.05, random_state=42)
src=partialtrainData["train.SRC"].to_numpy()
trg=partialtrainData["train.EXR"].to_numpy()


In [6]:
print(src.shape)
print(trg.shape)


(122822,)
(122822,)


In [7]:
def save_data(input_sequences, target_sequences, input_filename, target_filename):
    # Convert to tensors and save them
    torch.save(input_sequences, input_filename)
    torch.save(target_sequences, target_filename)


In [8]:
# Build vocab function (as described earlier)
def build_vocab(sentences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<unk>", filters="")
    tokenizer.fit_on_texts(sentences)
    
    vocab = tokenizer.word_index
    vocab["<pad>"] = 0
    vocab["<sos>"] = len(vocab) 
    vocab["<eos>"] = len(vocab) 
    
    tokenizer.word_index = vocab
    # print(tokenizer.word_index)
    return tokenizer

# Build vocabulary using the preprocessed orders (after tokenizing)
input_vocab = build_vocab([preprocess_order(order) for order in src])
output_vocab = build_vocab([preprocess_order(order) for order in trg])


# Function to process data into numerical sequences
def process_data(sentences, vocab, is_output=False):
    sequences = []
    for sentence in sentences:
        tokens = preprocess_order(sentence)  # Apply the preprocessing function
        if is_output:
            tokens = ["<sos>"] + tokens + ["<eos>"]  # Add <sos> and <eos> for output sequences
        indices = [vocab[token] for token in tokens if token in vocab]  # Ensure token exists in vocab
        sequences.append(indices)
    
    # Pad the sequences to make them uniform length
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post', value=vocab["<pad>"])

# Convert src_train and trg_train to numerical sequences
input_sequences = process_data(src, input_vocab.word_index)
output_sequences = process_data(trg, output_vocab.word_index, is_output=True)

# Print the processed and padded sequences
print(input_sequences.shape)
print(output_sequences.shape)


(122822, 20)
(122822, 32)


In [9]:
vocab_size = len(input_vocab.word_index) 
print(f"input Vocabulary Size: {vocab_size}")
vocab_size = len(output_vocab.word_index) 
print(f"output Vocabulary Size: {vocab_size}")


input Vocabulary Size: 295
output Vocabulary Size: 181


In [10]:
# Get the inverse mapping of the vocab (index -> word)
index_to_word = {index: word for word, index in input_vocab.word_index.items()}

# Filter out padding (0) from the sequence and map indices to words
words_in_sequence = [index_to_word.get(index, "<unk>") for index in input_sequences[0] if index != input_vocab.word_index["<pad>"]]


print("Words in input_sequences[0] without padding:", words_in_sequence)

Words in input_sequences[0] without padding: ['id', 'like', 'three', 'large', 'pies', 'pestos', 'yellow', 'peppers']


In [11]:
input_sequences = torch.from_numpy(input_sequences)
output_sequences=torch.from_numpy(output_sequences)

# Assuming `input_data` and `target_data` are your raw datasets
save_data(input_sequences, output_sequences, 'input_sequences.pt', 'target_sequences.pt')

del input_sequences
del output_sequences

In [12]:
class CustomDataset(Dataset):
    def __init__(self, input_file, target_file):
        # Load the preprocessed data
        self.input_data = torch.load(input_file)
        self.target_data = torch.load(target_file)
        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        # Return a single sample (input, target) pair
        input_sample = self.input_data[idx]
        target_sample = self.target_data[idx]
        return input_sample, target_sample

# Create the Dataset object
dataset = CustomDataset('input_sequences.pt', 'target_sequences.pt')


/tmp/ipykernel_13/3440725130.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.input_data = torch.load(input_file)
/tmp/ipykernel_13/3440725130.py:5: FutureWarning: Y

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embed_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)
        # Concatenate forward and backward hidden states
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1).unsqueeze(0)
        cell = torch.cat((cell[-2], cell[-1]), dim=1).unsqueeze(0)
        return outputs, hidden, cell

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embed_size, hidden_size, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)  # Add batch dimension
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        predictions = self.fc(output.squeeze(1))
        return predictions, hidden, cell

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)

        # Encode source
        _, hidden, cell = self.encoder(source)

        # First input to decoder is <sos>
        decoder_input = target[:, 0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[:, t, :] = output
            # Use teacher forcing
            decoder_input = target[:, t] if torch.rand(1).item() < teacher_forcing_ratio else output.argmax(1)

        return outputs

In [16]:
print("Input vocab size:", len(input_vocab.word_index))
print("Output vocab size:", len(output_vocab.word_index))


Input vocab size: 295
Output vocab size: 181


In [19]:
device = torch.device("cpu")

# Hyperparameters
input_vocab_size = len(input_vocab.word_index)
output_vocab_size = len(output_vocab.word_index)
embed_size = 256
hidden_size = 512
num_layers = 1
learning_rate = 0.001
# batch_size = input_sequences.shape[0]
# batch_size = 256
epochs = 10

# Create the DataLoader for batching
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
clip=0.1

# Initialize the model, optimizer, and loss function
encoder = Encoder(input_vocab_size, embed_size, hidden_size, num_layers).to(device)
decoder = Decoder(output_vocab_size, embed_size, hidden_size * 2, num_layers).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=output_vocab.word_index["<pad>"])

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss=0

    for input_batch, target_batch in dataloader:
        # Move data to the correct device
        source = input_batch.to(device)
        target = target_batch.to(device)
        
        optimizer.zero_grad()
    
        # Move data to device
        # source = input_sequences.to(device)
        # target = output_sequences.to(device)
    
        # Forward pass
        outputs = model(source, target)
    
        # Reshape for calculating loss
        outputs = outputs[:, 1:].reshape(-1, outputs.shape[2])
        target = target[:, 1:].reshape(-1)
    
        target = target.long()
        loss = criterion(outputs, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss+=loss.item()

    if(epoch_loss<=10**-4):
        break
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")


Epoch [1/10], Loss: 284.2593
Epoch [2/10], Loss: 6.5323
Epoch [3/10], Loss: 3.4658
Epoch [4/10], Loss: 3.4314
Epoch [5/10], Loss: 2.4000
Epoch [6/10], Loss: 3.1380
Epoch [7/10], Loss: 2.1699
Epoch [8/10], Loss: 1.9265
Epoch [9/10], Loss: 1.7594
Epoch [10/10], Loss: 1.9494


In [21]:
devData = pd.read_json("/kaggle/input/PIZZA_dev.json", lines=True)
src_dev=devData["dev.SRC"].to_numpy()
trg_dev=devData["dev.EXR"].to_numpy()

input_dev_sequences = process_data(src_dev, input_vocab.word_index)
output_dev_sequences = process_data(trg_dev, output_vocab.word_index, is_output=True)


In [29]:
import torch

def evaluate(src, target):
    total_correct = 0
    model.eval()

    # Forward pass through the model on the input data
    with torch.no_grad():
        for source, target_batch in zip(src, target):
            # Convert numpy arrays to PyTorch tensors
            source = torch.tensor(source, dtype=torch.long).to(device)
            target_batch = torch.tensor(target_batch, dtype=torch.long).to(device)
            
            # # Debug shapes
            # print(f"source shape: {source.shape}")
            # print(f"target_batch shape: {target_batch.shape}")

            # Ensure proper batch dimensions
            if source.dim() == 1:
                source = source.unsqueeze(0)  # Add batch dimension if missing
            if target_batch.dim() == 1:
                target_batch = target_batch.unsqueeze(0)

            # Forward pass
            output = model(source, target_batch)
            
            # Reshape for comparison
            output = output[:, 1:].reshape(-1, output.shape[2])
            target_batch = target_batch[:, 1:].reshape(-1)

            # Mask padding tokens
            mask = target_batch != 0
            valid_output = output[mask]
            valid_target = target_batch[mask]

            # Get predicted tokens
            _, predicted = torch.max(valid_output, 1)

            # Calculate correct predictions
            if torch.all(predicted == valid_target):
                total_correct += 1

    # Calculate overall accuracy
    accuracy = total_correct / len(src)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [30]:
evaluate(input_dev_sequences,output_dev_sequences)

Test Accuracy: 6.03%


In [ ]:
# # Save the model's state dictionary
# torch.save(model.state_dict(), "seq2seq_state_dict.pth")
# print("Model state dictionary saved!")